In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import re
import string

DT = pd.read_csv(r'C:\Users\MY\Downloads\Datasets\DisasterTweets\train.csv')
DT.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [2]:
DT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [3]:
DT.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [4]:
DT['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [5]:
Counter(DT['location']).most_common()[1]

('USA', 104)

In [6]:
DT['location'].mode()[0]

'USA'

In [7]:
MaxLoc = DT['location'].mode()[0]

In [8]:
DT['location'].fillna(MaxLoc, inplace = True)

In [9]:
Counter(DT['keyword']).most_common()[1]

('fatalities', 45)

In [10]:
def URL(tweet):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', tweet)

def HTML(tweet):
    html = re.compile(r'<.*?>')
    return html.sub(r'', tweet)

def emoji(tweet):
    emojis = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emojis.sub(r'', tweet)

def punct(tweet):
    puncts = tweet.maketrans('', '', string.punctuation)
    return tweet.translate(puncts)

def case(tweet):
    tweet = tweet.lower()
    return tweet

In [11]:
DT['text'] = DT['text'].apply(lambda x : URL(x))
DT['text'] = DT['text'].apply(lambda x : HTML(x))
DT['text'] = DT['text'].apply(lambda x : emoji(x))
DT['text'] = DT['text'].apply(lambda x : punct(x))
DT['text'] = DT['text'].apply(lambda x : case(x))

In [12]:
from rake_nltk import Rake

extractor = Rake()

def fillk(w):
    if(pd.isnull(w['keyword'])):
        extractor.extract_keywords_from_text(w['text'])
        key_extracted = extractor.get_ranked_phrases()[0]
        print(key_extracted)
        w['keyword'] = key_extracted 
        
    else:
        w = w
        
    return w

DT = DT.apply(fillk, axis = 1)            

earthquake may allah forgive us
forest fire near la ronge sask canada
residents asked
13000 people receive wildfires evacuation orders
wildfires pours
rockyfire update california hwy 20 closed
flood disaster heavy rain causes flash flooding
woods
emergency evacuation happening
im afraid
three people died
getting flooded hah wait
raining flooding florida tampabay tampa 18
bago myanmar
multi car crash breaking
whats
love fruits
summer
fast
goooooooaaaaaal
ridiculous
london
love skiing
wonderful day
looooool
wayi cant eat
nyc last week
love
cooool
like pasta
end
16yr old pkk suicide bomber
explode exploding kittens finally arrived gameofkittens explodingkittens  û
calgary police flood road closures
sismo detectado japìn 154107 seismic intensity 0 iwate miyagi jst
sirens everywhere
breaking isis claims responsibility
omg earthquake
typhoon  ûïhannaph  û  soudelor tropical cyclone warning issued
heat wave warning aa ayyo dei
saudi special forces headquarters killing 15 people
really lou

In [13]:
DT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7613 non-null   object
 2   location  7613 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [14]:
DT.isnull().sum()

id          0
keyword     0
location    0
text        0
target      0
dtype: int64

In [15]:
from sklearn.model_selection import train_test_split, GridSearchCV

X_train1, X_test1, y_train1, y_test1 = train_test_split(DT['text'].tolist(), DT['target'].tolist(), 
                                                        test_size = 0.2, random_state = 1)
X_train2, X_test2, y_train2, y_test2 = train_test_split(DT['text'].tolist(), DT['target'].tolist(),
                                                        test_size = 0.2, random_state = 1)

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
stopwords = stopwords.words('english')

countvec = CountVectorizer(strip_accents = 'ascii', stop_words = stopwords, ngram_range = (1, 2))
tfidf = TfidfVectorizer(strip_accents = 'ascii', stop_words = stopwords, ngram_range = (1, 2))

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

countfeatures = countvec.fit_transform(X_train1)
counttestfeatures = countvec.transform(X_test1)

model = LogisticRegression()
model.fit(countfeatures, y_train1)
predictions = model.predict(counttestfeatures)
ac = accuracy_score(y_test1, predictions)
print(ac)

0.8030203545633617


In [18]:
tfidffeatures = tfidf.fit_transform(X_train2)
tfidftestfeatures = tfidf.transform(X_test2)

model.fit(tfidffeatures, y_train2)
predictions = model.predict(tfidftestfeatures)
ac = accuracy_score(y_test2, predictions)
print(ac)

0.8049901510177282


In [19]:
y = DT['target']

In [20]:
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

#modeltest = RandomForestClassifier()
#modelc = svm.SVC(kernel = 'linear', C = 1.0).fit(X_train1, y_train1)
#predictions = modelc.predict(X_test1)
#ac = accuracy_score(predictions, y_test1)
#print(ac)

In [21]:
#from sklearn import svm
#from sklearn.metrics import accuracy_score
#modelt = svm.SVC(kernel = 'linear', C = 1.0).fit(X_train2, y_train2)
#predictions = modelt.predict(X_test2)
#ac = accuracy_score(predictions, y_test2)
#print(ac)

In [22]:
test = pd.read_csv(r'C:\Users\MY\Downloads\Datasets\DisasterTweets\test.csv')
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [23]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3263 non-null   int64 
 1   keyword   3237 non-null   object
 2   location  2158 non-null   object
 3   text      3263 non-null   object
dtypes: int64(1), object(3)
memory usage: 102.1+ KB


In [24]:
test.isnull().sum()

id             0
keyword       26
location    1105
text           0
dtype: int64

In [25]:
Counter(test['location']).most_common(2)

[(nan, 1105), ('New York', 38)]

In [26]:
test['location'].mode()[0]

'New York'

In [27]:
MaxLoct = test['location'].mode()[0]

In [28]:
test['location'].fillna(MaxLoct, inplace = True) 

In [29]:
test['text'] = test['text'].apply(lambda x : URL(x))
test['text'] = test['text'].apply(lambda x : HTML(x))
test['text'] = test['text'].apply(lambda x : emoji(x))
test['text'] = test['text'].apply(lambda x : punct(x))
test['text'] = test['text'].apply(lambda x : case(x))

In [30]:
def fillkt(t):
    if(pd.isnull(t['keyword'])):
        extractor.extract_keywords_from_text(t['text'])
        key_extracted1 = extractor.get_ranked_phrases()[:1]
        print(key_extracted1)
        t['keyword'] = key_extracted1
        
    else:
        t = t
        
    return t

test = test.apply(fillkt, axis = 1)

['terrible car crash']
['different cities stay safe everyone']
['spot pond geese']
['apocalypse lighting spokane wildfires']
['typhoon soudelor kills 28']
['shakingits']
['theyd probably still show']
['hey']
['nice hat']
['fuck']
['dont like cold']
['nooooooooo dont']
['dont tell']
[]
['awesome']
['ebola ebolaoutbreak ebola virus birmingham ala firefighters quarantined']
['malaysian pm confirms debris']
['possible ebola case washington times']
['turkey army trench released harun ìàekdar msoreo']
['derailed please look']
['issuicide car bombing']
['earthquake safety los angeles \x89 ûò safety fasteners xrwn']
['bombed around 20000k still without power']
['green line derailment']
['meg issues hazardous weather outlook hwo']
['municipal emergency plan yycstorm']


In [31]:
test.isnull().sum()

id          0
keyword     0
location    0
text        0
dtype: int64

In [32]:
countfeatures = countvec.fit_transform(DT['text'].tolist())
testfeatures = countvec.transform(test['text'].tolist())

model.fit(countfeatures, DT['target'].tolist())
predictions = model.predict(testfeatures)
print(predictions)

[1 1 1 ... 1 1 1]


In [33]:
ID = pd.Series(test['id'])
targets = pd.Series(predictions)

data = pd.concat([ID, targets], axis = 1)

In [34]:
df = pd.DataFrame(data).reset_index()
df.columns = ['Index', 'id', 'target']
df = df.drop(['Index'], axis = 1)
df.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,0
4,11,1


In [35]:
df['target'].value_counts()

0    2174
1    1089
Name: target, dtype: int64

In [37]:
df.to_csv(r'C:\Users\MY\Downloads\Datasets\DisasterTweets\LR10.csv', index = False)